# Quantum Supremacy

### Imports Go Here

In [3]:
from qiskit import Aer
import numpy as np
import getpass
from qiskit_ionq_provider import IonQProvider 
key = str(np.genfromtxt('ionQkey.txt',dtype='str'))
provider = IonQProvider(token=key)
from qiskit import QuantumCircuit
import time, sys

### Functions Go Here

In [6]:
def get_players():
    N = 0
    while N < 2:
        N = int(input('How many players are there?\n Enter Number: '))
        
        if N < 2:
            delay_print('Please have at least two players')
    return N

def get_init_state(N):
    qc = QuantumCircuit(N)
    init_string = ''
    for i in np.arange(N):
        state_valid = False
        while not state_valid: 
            delay_print("Player %d, select your initial state:" % (i+1))
            delay_print(' 0: |0>')
            delay_print(' 1: |1>')
            delay_print(' 2: (|0> + |1>)/sqrt(2)')
            state = getpass.getpass('')
            print('')
            
            if (state != '0') and (state != '1') and (state != '2'):
                delay_print('Please input a valid state')
            else:
                state_valid=True
                
        init_string += state

    for i, s in enumerate(init_string):
        if s=='1':
            qc.x(i)
        if s=='2':
            qc.h(i)    
    return qc

def get_gate_and_target(N):
    
    # Get gate to add
    delay_print("  What gate would you like to add?")
    delay_print("    0:X      (1-qubit)", dt=0.01)
    delay_print("    1:Y      (1-qubit)", dt=0.01)
    delay_print("    2:Z      (1-qubit)", dt=0.01)
    delay_print("    3:H      (1-qubit)", dt=0.01)
    delay_print("    4:CNOT   (2-qubit)", dt=0.01)
    delay_print("    5:CPHASE (2-qubit)", dt=0.01)
    delay_print("    6:Swap   (2-qubit)", dt=0.01)

    while True:
        try:
            gate = int(input('    Input: '))
        except ValueError:
            delay_print(    "Sorry, didn't understand that.")
            continue
        else:
            if gate<7:
                break
            print("    Sorry, choose a gate between 0 and 6.")

    # Get target(s) to add gate to
    delay_print("  Where would you like to add this gate?")
    while True:
        try:
            target = list(map(int, input("    Input (i or i,j): ").split(',')))
            target = [t-1 for t in target]
        except ValueError:
            delay_print(    "Sorry, didn't understand that!")
            continue
        else:

            # Ask for another input if bad qubit index
            if max(target)+1 > N or min(target)+1<1:
                delay_print("    Sorry, use a qubit index between 1 and %d!" % N)
                continue

            # Use this input if we have right number of targets
            if (gate<4 and len(target)==2) or (gate>3 and len(target)==1) or len(target)>2 or len(target)==0:
                delay_print("    Sorry, use the right number of qubits!")
                continue

            break

    return gate, target

def davids_gameplay_loop(N_players, N_rounds=1):
    
    qc = QuantumCircuit(N_players)
    
    for i in range(N_rounds):

        delay_print("-"*30 + "Round %d!"%(i+1) + "-"*30)

        for j in range (N_players):

            delay_print("Hello player %d!\n" % (j+1))

            gate, target = get_gate_and_target(N_players)

            if gate == 0:
                qc.x(target[0])
            elif gate == 1:
                qc.y(target[0])
            elif gate == 2:
                qc.z(target[0])
            elif gate == 3:
                qc.h(target[0])
            elif gate == 4:
                qc.cx(target[0],target[1])
            elif gate == 5:
                qc.cz(target[0],target[1])
            elif gate == 6:
                qc.SwapGate(target[0],target[1])

            print(qc)

            delay_print('\n'+"-"*40)
            
    return qc

def game_end(N, qc_init, qc_game, shots=100, backend='ionq_simulator'):
    qc = qc_init + qc_game
    qc.measure_all()
    backend = provider.get_backend(backend)
    job = backend.run(qc, shots=shots)
    result = job.result()
    counts_dict = result.data()['counts']
    
    player_counts = np.zeros(N)
    for key in counts_dict:
        for i, s, in enumerate(key):
            player_counts[i] += int(s) * counts_dict[key]
    player_p1 = player_counts / shots
    
    return player_p1, np.argmax(player_p1)

def delay_print(s, dt=0.00):
    for c in s:
        sys.stdout.write(c)
        sys.stdout.flush()
        time.sleep(dt)
    sys.stdout.write('\n')
    sys.stdout.flush()

def main():
    
    f = open("title.txt", "r", encoding="utf8")
    delay_print(f.read(), dt=0.005)
    delay_print(80*'-', dt=0.005)
    s0 = " In this game, the players are qubits."
    s1 = " Your goal is to maximize your z polarizaiton on the Bloch sphere."
    s2 = " Players will pick the initial state of their qubit, unknown to the other players."
    s3 = " Then, players will consecutively add gates to a circuit, rotating or entangling any qubits."
    s4 = " After every new gate is added, a measurement bitstring will be given, to get a glimpse of the current state."
    s5 = " The player with the maximum z-polarization at the end wins...good luck!~"
    delay_print("Welcome to Quantum Supremacy!"+s1+s2+s3+s4+s5, dt=0.05)

    N = get_players()
    delay_print(80*'-', dt=0.005)
    qc_init = get_init_state(N)
    delay_print(80*'-', dt=0.005)
    delay_print('Let the game begin! We will begin the adding of gates now...', dt=0.005)
    qc_game = davids_gameplay_loop(N)
    
    delay_print('Measuring...', dt=0.005)
    p1_vec, winner_ind = game_end(N, qc_init, qc_game)
        
    delay_print('Final Circuit:')
    print(qc_init + qc_game)
    delay_print('')
    
    delay_print('Results:')
    print(p1_vec)
    delay_print('Player ' + str(winner_ind + 1) + ' is the winner!')

### Play Here

# main()

In [5]:
main()

﻿    ░██████╗░██╗░░░██╗░█████╗░███╗░░██╗████████╗██╗░░░██╗███╗░░░███╗
    ██╔═══██╗██║░░░██║██╔══██╗████╗░██║╚══██╔══╝██║░░░██║████╗░████║
    ██║██╗██║██║░░░██║███████║██╔██╗██║░░░██║░░░██║░░░██║██╔████╔██║
    ╚██████╔╝██║░░░██║██╔══██║██║╚████║░░░██║░░░██║░░░██║██║╚██╔╝██║
    ░╚═██╔═╝░╚██████╔╝██║░░██║██║░╚███║░░░██║░░░╚██████╔╝██║░╚═╝░██║
    ░░░╚═╝░░░░╚═════╝░╚═╝░░╚═╝╚═╝░░╚══╝░░░╚═╝░░░░╚═════╝░╚═╝░░░░░╚═╝

    ░██████╗██╗░░░██╗██████╗░██████╗░███████╗███╗░░░███╗░█████╗░░█████╗░██╗░░░██╗
    ██╔════╝██║░░░██║██╔══██╗██╔══██╗██╔════╝████╗░████║██╔══██╗██╔══██╗╚██╗░██╔╝
    ╚█████╗░██║░░░██║██████╔╝██████╔╝█████╗░░██╔████╔██║███████║██║░░╚═╝░╚████╔╝░
    ░╚═══██╗██║░░░██║██╔═══╝░██╔══██╗██╔══╝░░██║╚██╔╝██║██╔══██║██║░░██╗░░╚██╔╝░░
    ██████╔╝╚██████╔╝██║░░░░░██║░░██║███████╗██║░╚═╝░██║██║░░██║╚█████╔╝░░░██║░░░
    ╚═════╝░░╚═════╝░╚═╝░░░░░╚═╝░░╚═╝╚══════╝╚═╝░░░░░╚═╝╚═╝░░╚═╝░╚════╝░░░░╚═╝░░░
--------------------------------------------------------------------------------
Welcome to 